# Talbot Carpet Raw Loader (B4)
Downloads the Zenodo archive `10.5281/zenodo.14002229`, extracts the HDF5/TIFF files,
re-averages the intensity stack, and reproduces the FFT z-slice with ±1⁄14 peaks.

In [ ]:
import requests, tarfile, io, os, numpy as np, h5py, matplotlib.pyplot as plt
doi = "10.5281/zenodo.14002229"
# ▼ user fills in downloading & analysis code here ▼